### Setup your environnement variables

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["UIFORM_API_KEY"] = "YOUR API KEY" # go to https://uiform.com to create your API Key

In [2]:
from dotenv import load_dotenv
assert load_dotenv("../.env.production") # Load environment variables from a .env file

# Get started

In [6]:
from uiform import UiForm
uiclient = UiForm()
uiclient.models.list()

['claude-3-5-sonnet-latest',
 'claude-3-5-sonnet-20241022',
 'claude-3-opus-20240229',
 'claude-3-sonnet-20240229',
 'claude-3-haiku-20240307',
 'gpt-4o',
 'gpt-4o-mini',
 'chatgpt-4o-latest',
 'gpt-4o-2024-11-20',
 'gpt-4o-2024-08-06',
 'gpt-4o-2024-05-13',
 'gpt-4o-mini-2024-07-18',
 'o1-2024-12-17',
 'o1-mini-2024-09-12',
 'grok-2-vision-1212',
 'grok-2-1212',
 'gemini-2.0-flash-exp',
 'gemini-1.5-flash-8b',
 'gemini-1.5-flash',
 'gemini-1.5-pro']

In [11]:
from io import BytesIO
empty_bytesio = BytesIO(b"")
empty_bytesio.name = "empty.txt" 
            

doc_msg = uiclient.documents.create_messages(
    document = empty_bytesio,
)

In [12]:
doc_msg.messages

[{'role': 'user', 'content': [{'text': '', 'type': 'text'}]}]

# Example 1 - Analyze informations from a document

#### Option A - You use our preprocess endpoint and openAI client

In [3]:
import json
from uiform import UiForm, Schema
from openai import OpenAI

with open("freight/schema.json", "r") as f:
    json_schema = json.load(f)


uiclient = UiForm()
doc_msg = uiclient.documents.create_messages(
    document = "freight/booking_confirmation.jpg",
)

schema_obj = Schema(
    json_schema = json_schema
)


# Now you can use your favorite model to analyze your document
client = OpenAI()
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini", 
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format=schema_obj.inference_pydantic_model
)

In [4]:
# Validate the response against the original schema if you want to remove the reasoning fields
from uiform._utils.json_schema import filter_reasoning_fields_json
assert completion.choices[0].message.content is not None
extraction = schema_obj.pydantic_model.model_validate(
     filter_reasoning_fields_json(completion.choices[0].message.content)
)

#### You can also add some preprocessing operations to find informations that the LLM struggles to parse

In [20]:
text_operations = {
    'regex_instructions' : [
        {
            "name": "vat_number",
            "pattern": r"[Ff][Rr]\s*(\d\s*){11}",
            "description": "VAT number in the format XX999999999"
        }
    ]
}

doc_msg = uiclient.documents.create_messages(
    document = "freight/booking_confirmation.jpg",
    text_operations=text_operations
)

#### Option B - Using UiForm `extract` endpoint

In [3]:
import json
from uiform.client import UiForm

uiclient = UiForm()
completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini",
    temperature=0
)

print(completion.choices[0].message.parsed)

booking_id='SHIP-001' payment=TransportPriceData(total_price=1500.0, currency='EUR') client=ClientData(company_name='ACME Corporation', VAT_number='GB123456789', city='London', postal_code='WC2N 5DU', country='GB', code='BC-67890', email='client@acme.com') shipments=[ShipmentData(shipment_id='SHIP-001', sender=SenderData(company_name='ACME Corporation', address=AddressDataSimple(city='London', postal_code='WC2N 5DU', country='GB', line1='123 Elm Street', line2='Suite 500'), phone_number='+44 20 7946 0958', email_address='client@acme.com', pickup_datetime=PickupDatetimeData(date='2023-02-05', start_time='08:00:00', end_time='12:00:00'), observations=None), recipient=RecipientData(company_name='Beta Industries', address=AddressDataSimple(city='Munich', postal_code='80331', country='DE', line1='789 Pine Street', line2=None), phone_number='+49 89 123456', email_address=None, delivery_datetime=DeliveryDatetimeData(date='2023-03-05', start_time='10:00:00', end_time='16:00:00'), observations=

In [13]:
import json
from uiform.client import UiForm

uiclient = UiForm()
completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini",
    temperature=0
)

print(completion.choices[0].message.content)

{"reasoning___root": "Analysis shows:\n- Clear booking confirmation document from ACME Corporation.\n- Contains 1 distinct shipment under single booking reference.\n- Client identified as ACME Corporation from letterhead.\n- Total price and payment terms clearly stated.\n- All required signatures present.", "booking_id": "SHIP-001", "payment": {"total_price": 1500, "currency": "EUR"}, "client": {"company_name": "ACME Corporation", "reasoning___VAT_number": "Found VAT number GB123456789 in header next to company name.", "VAT_number": "GB123456789", "city": "London", "postal_code": "WC2N 5DU", "country": "GB", "code": "SHIP-001", "email": "client@acme.com"}, "reasoning___shipments": "Shipment Analysis:\n1: 10 pallets, 1.44 M3, 400 kg (UN 1202, Class 3 - Liquid flammable)", "shipments": [{"shipment_id": "SHIP-001", "reasoning___sender": "I have found within the documents the following information about the sender:\n- Sender company name: ACME Corporation\n- Sender address: LINE1: 123 Elm 

In [14]:
print(completion.choices[0].message.parsed)

booking_id='SHIP-001' payment=TransportPriceData(total_price=1500.0, currency='EUR') client=ClientData(company_name='ACME Corporation', VAT_number='GB123456789', city='London', postal_code='WC2N 5DU', country='GB', code='SHIP-001', email='client@acme.com') shipments=[ShipmentData(shipment_id='SHIP-001', sender=SenderData(company_name='ACME Corporation', address=AddressDataSimple(city='London', postal_code='WC2N 5DU', country='GB', line1='123 Elm Street', line2='Suite 500'), phone_number='+44 20 7946 0958', email_address='client@acme.com', pickup_datetime=PickupDatetimeData(date='2023-02-05', start_time='08:00:00', end_time='12:00:00'), observations=None), recipient=RecipientData(company_name='Beta Industries', address=AddressDataSimple(city='Munich', postal_code='80331', country='DE', line1='789 Pine Street', line2=None), phone_number='+49 89 123456', email_address=None, delivery_datetime=DeliveryDatetimeData(date='2023-03-05', start_time='10:00:00', end_time='16:00:00'), observations=

In [15]:
completion.choices[0].message.parsed.model_dump()

{'booking_id': 'SHIP-001',
 'payment': {'total_price': 1500.0, 'currency': 'EUR'},
 'client': {'company_name': 'ACME Corporation',
  'VAT_number': 'GB123456789',
  'city': 'London',
  'postal_code': 'WC2N 5DU',
  'country': 'GB',
  'code': 'SHIP-001',
  'email': 'client@acme.com'},
 'shipments': [{'shipment_id': 'SHIP-001',
   'sender': {'company_name': 'ACME Corporation',
    'address': {'city': 'London',
     'postal_code': 'WC2N 5DU',
     'country': 'GB',
     'line1': '123 Elm Street',
     'line2': 'Suite 500'},
    'phone_number': '+44 20 7946 0958',
    'email_address': 'client@acme.com',
    'pickup_datetime': {'date': '2023-02-05',
     'start_time': '08:00:00',
     'end_time': '12:00:00'},
    'observations': None},
   'recipient': {'company_name': 'Beta Industries',
    'address': {'city': 'Munich',
     'postal_code': '80331',
     'country': 'DE',
     'line1': '789 Pine Street',
     'line2': None},
    'phone_number': '+49 89 123456',
    'email_address': None,
    'de

## Appendix A - Use text as modality

In [22]:
from uiform.client import UiForm

uiclient = UiForm()

completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document="freight/booking_confirmation.jpg",
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    modality='text'
)

print(completion.choices[0].message.parsed)

booking_id='SHIP-001' payment=TransportPriceData(total_price=1500.0, currency='EUR') client=ClientData(company_name='ACME Corporation', VAT_number=None, city='London', postal_code='WC2N S5DU', country='GB', code=None, email='client@acme.com') shipments=[ShipmentData(shipment_id='SHIP-001', sender=SenderData(company_name='ACME Corporation', address=AddressDataSimple(city='Manchester', postal_code='M1 4WP', country='GB', line1='456 Oak Avenue', line2='Floor 3 Suite 500'), phone_number='+442079460958', email_address='client@acme.com', pickup_datetime=PickupDatetimeData(date='2023-02-05', start_time='08:00:00', end_time='12:00:00'), observations=None), recipient=RecipientData(company_name='Beta Industries', address=AddressDataSimple(city='Munich', postal_code='80331', country='DE', line1='789 Pine Street', line2=None), phone_number='+4989123456', email_address=None, delivery_datetime=DeliveryDatetimeData(date='2023-03-05', start_time='10:00:00', end_time='16:00:00'), observations=None), go

## Appendix B - Add text operations

In [ ]:
from uiform.client import UiForm

uiclient = UiForm()

text_operations = {
    'regex_instructions' : [
        {
            "name": "vat_number",
            "pattern": r"[Ff][Rr]\s*(\d\s*){11}",
            "description": "VAT number in the format XX999999999"
        }
    ]
}

completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    text_operations=text_operations
)

print(completion.choices[0].message.parsed)